Trains and Evaluates the IndianPines network using a feed dictionary
========================================

In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import time
import numpy as np
def dummy_npwarn_decorator_factory():
  def npwarn_decorator(x):
    return x
  return npwarn_decorator
np._no_nep50_warning = getattr(np, '_no_nep50_warning', dummy_npwarn_decorator_factory)
from six.moves import xrange  # pylint: disable=redefined-builtin
import tensorflow.compat.v1 as tf
tf.compat.v1.disable_v2_behavior()
print (tf.__version__)
import os
import IndianPinesCNN 
import patch_size
# import IndianPines_data_set as input_data
import Spatial_dataset as input_data


Instructions for updating:
non-resource variables are not supported in the long term
2.19.0


Declare model parameters as external flags
-------------------------------------

In [2]:
# flags = tf.app.flags
# FLAGS = flags.FLAGS
# flags.DEFINE_float('learning_rate', 0.01, 'Initial learning rate.')
# flags.DEFINE_integer('max_steps', 4000, 'Number of steps to run trainer.')
# flags.DEFINE_integer('conv1', 500, 'Number of filters in convolutional layer 1.')
# flags.DEFINE_integer('conv2', 100, 'Number of filters in convolutional layer 2.')
# flags.DEFINE_integer('hidden1', 200, 'Number of units in hidden layer 1.')
# flags.DEFINE_integer('hidden2', 84, 'Number of units in hidden layer 2.')
# flags.DEFINE_integer('batch_size', 100, 'Batch size.  '
#                      'Must divide evenly into the dataset sizes.')
# # flags.DEFINE_string('train_dir', '1.mat', 'Directory to put the training data.')

learning_rate = 0.01  # Initial learning rate
max_steps = 4000  # Number of steps to run trainer
conv1 = 500  # Number of filters in convolutional layer 1
conv2 = 100  # Number of filters in convolutional layer 2
hidden1 = 200  # Number of units in hidden layer 1
hidden2 = 84  # Number of units in hidden layer 2
batch_size = 100  # Batch size

# import argparse

# parser = argparse.ArgumentParser()
# parser.add_argument('--learning_rate', type=float, default=0.01, help='Initial learning rate.')
# parser.add_argument('--max_steps', type=int, default=4000, help='Number of steps to run trainer.')
# parser.add_argument('--conv1', type=int, default=500, help='Number of filters in convolutional layer 1.')
# parser.add_argument('--conv2', type=int, default=100, help='Number of filters in convolutional layer 2.')
# parser.add_argument('--hidden1', type=int, default=200, help='Number of units in hidden layer 1.')
# parser.add_argument('--hidden2', type=int, default=84, help='Number of units in hidden layer 2.')
# parser.add_argument('--batch_size', type=int, default=100, help='Batch size.')

# FLAGS, _ = parser.parse_known_args()


In [3]:
learning_rate = 0.01
num_epochs = 20
max_steps = 4000
IMAGE_SIZE = patch_size.patch_size
conv1 = 500
conv2 = 100
fc1 = 200,
fc2 = 84
batch_size = 100
TRAIN_FILES = 8
TEST_FILES = 4
DATA_PATH = os.path.join(os.getcwd(),"Data")

In [4]:
def placeholder_inputs(batch_size):
    """Generate placeholder variables to represent the input tensors.
    These placeholders are used as inputs by the rest of the model building
    code and will be fed from the downloaded data in the .run() loop, below.
    Args:
    batch_size: The batch size will be baked into both placeholders.
    Returns:
    images_placeholder: Images placeholder.
    labels_placeholder: Labels placeholder.
    """
    # Note that the shapes of the placeholders match the shapes of the full
    # image and label tensors, except the first dimension is now batch_size
    # rather than the full size of the train or test data sets.
    images_placeholder = tf.placeholder(tf.float32, shape=(batch_size, IndianPinesCNN
                                                           .IMAGE_PIXELS))
    labels_placeholder = tf.placeholder(tf.int32, shape=(batch_size))
    return images_placeholder, labels_placeholder


In [5]:
def fill_feed_dict(data_set, images_pl, labels_pl):
    """Fills the feed_dict for training the given step.
    A feed_dict takes the form of:
    feed_dict = {
      <placeholder>: <tensor of values to be passed for placeholder>,
      ....
    }
    Args:
    data_set: The set of images and labels, from input_data.read_data_sets()
    images_pl: The images placeholder, from placeholder_inputs().
    labels_pl: The labels placeholder, from placeholder_inputs().
    Returns:
    feed_dict: The feed dictionary mapping from placeholders to values.
    """
    # Create the feed_dict for the placeholders filled with the next
    # `batch size ` examples.
    images_feed, labels_feed = data_set.next_batch(batch_size)
    feed_dict = {
      images_pl: images_feed,
      labels_pl: labels_feed,
    }
    return feed_dict

In [6]:
def do_eval(sess,
            eval_correct,
            images_placeholder,
            labels_placeholder,
            data_set):
    """Runs one evaluation against the full epoch of data.
    Args:
    sess: The session in which the model has been trained.
    eval_correct: The Tensor that returns the number of correct predictions.
    images_placeholder: The images placeholder.
    labels_placeholder: The labels placeholder.
    data_set: The set of images and labels to evaluate, from
      input_data.read_data_sets().
    """
    # And run one epoch of eval.
    true_count = 0  # Counts the number of correct predictions.
    steps_per_epoch = data_set.num_examples // batch_size
    num_examples = steps_per_epoch * batch_size
    for step in xrange(steps_per_epoch):
        feed_dict = fill_feed_dict(data_set,
                                   images_placeholder,
                                   labels_placeholder)
        true_count += sess.run(eval_correct, feed_dict=feed_dict)
    precision = true_count / num_examples
    print('  Num examples: %d  Num correct: %d  Precision @ 1: %0.04f' %
        (num_examples, true_count, precision))


In [ ]:
def add_DataSet(first,second):
    temp_image = np.concatenate((first.images,second.images),axis=0)
    temp_labels = np.concatenate((first.labels,second.labels),axis=0)
    temp_image = temp_image.reshape(temp_image.shape[0],IMAGE_SIZE,IMAGE_SIZE,220)
    temp_image = np.transpose(temp_image,(0,3,1,2))
    temp_labels = np.transpose(temp_labels)
    return input_data.DataSet(temp_image,temp_labels)

In [ ]:
def run_training():
    """Train MNIST for a number of steps."""
    # Get the sets of images and labels for training, validation, and
    # test on IndianPines.
    
    """Concatenating all the training and test mat files"""
    for i in range(TRAIN_FILES):
        data_sets = input_data.read_data_sets(os.path.join(DATA_PATH, 'Train_'+str(IMAGE_SIZE)+'_'+str(i+1)+'.mat'), 'train')
        if(i==0):
            Training_data = data_sets
            continue
        else:
            Training_data = add_DataSet(Training_data,data_sets)
            
    for i in range(TEST_FILES):
        data_sets = input_data.read_data_sets(os.path.join(DATA_PATH, 'Test_'+str(IMAGE_SIZE)+'_'+str(i+1)+'.mat'),'test')
        if(i==0):
            Test_data = data_sets
            continue
        else:
            Test_data = add_DataSet(Test_data,data_sets)
        
    # Tell TensorFlow that the model will be built into the default Graph.
    with tf.Graph().as_default():
    # Generate placeholders for the images and labels.
        images_placeholder, labels_placeholder = placeholder_inputs(batch_size)

        # Build a Graph that computes predictions from the inference model.
        logits = IndianPinesCNN.inference(images_placeholder,
                                 conv1,
                                 conv2,        
                                 hidden1,
                                 hidden2)

        # Add to the Graph the Ops for loss calculation.
        loss = IndianPinesCNN.loss(logits, labels_placeholder)

        # Add to the Graph the Ops that calculate and apply gradients.
        train_op = IndianPinesCNN.training(loss, learning_rate)

        # Add the Op to compare the logits to the labels during evaluation.
        eval_correct = IndianPinesCNN.evaluation(logits, labels_placeholder)

        # Build the summary operation based on the TF collection of Summaries.
    #    summary_op = tf.merge_all_summaries()

        # Add the variable initializer Op.
        init = tf.initialize_all_variables()

        # Create a saver for writing training checkpoints.
        saver = tf.train.Saver()

        # Create a session for running Ops on the Graph.
        sess = tf.Session()

        # Instantiate a SummaryWriter to output summaries and the Graph.
    #    summary_writer = tf.train.SummaryWriter(Ftrain_dir, sess.graph)

        # And then after everything is built:

        # Run the Op to initialize the variables.
        sess.run(init)

        # Start the training loop.
        for step in xrange(max_steps):
            start_time = time.time()

            # Fill a feed dictionary with the actual set of images and labels
            # for this particular training step.
            feed_dict = fill_feed_dict(Training_data,
                                     images_placeholder,
                                     labels_placeholder)

            # Run one step of the model.  The return values are the activations
            # from the `train_op` (which is discarded) and the `loss` Op.  To
            # inspect the values of your Ops or variables, you may include them
            # in the list passed to sess.run() and the value tensors will be
            # returned in the tuple from the call.
            _, loss_value = sess.run([train_op, loss],
                                   feed_dict=feed_dict)

            duration = time.time() - start_time

            # Write the summaries and print an overview fairly often.
            if step % 50 == 0:
            # Print status to stdout.
                print('Step %d: loss = %.2f (%.3f sec)' % (step, loss_value, duration))
            # Update the events file.
                # summary_str = sess.run(summary_op, feed_dict=feed_dict)
                # summary_writer.add_summary(summary_str, step)
                # summary_writer.flush()

            # Save a checkpoint and evaluate the model periodically.
            if (step + 1) % 1000 == 0 or (step + 1) == max_steps:
                saver.save(sess, 'model-spatial-CNN-'+str(IMAGE_SIZE)+'X'+str(IMAGE_SIZE)+'.ckpt', global_step=step)

            # Evaluate against the training set.
                print('Training Data Eval:')
                do_eval(sess,
                        eval_correct,
                        images_placeholder,
                        labels_placeholder,
                        Training_data)
                print('Test Data Eval:')
                do_eval(sess,
                        eval_correct,
                        images_placeholder,
                        labels_placeholder,
                        Test_data)
            # Evaluate against the validation set.
    #             print('Validation Data Eval:')
    #             do_eval(sess,
    #                     eval_correct,
    #                     images_placeholder,
    #                     labels_placeholder,
    #                     data_sets.validation)
    #             # Evaluate against the test set.
    #             print('Test Data Eval:')
    #             do_eval(sess,
    #                     eval_correct,
    #                     images_placeholder,
    #                     labels_placeholder,
    #                     data_sets.test)

In [ ]:
from sklearn.model_selection import KFold
import numpy as np

def run_training_with_cross_validation(k_folds=5):
    """
    Train the CNN model using K-Fold Cross-Validation.
    """
    # Load the full dataset (training + testing combined)
    full_data_images = []
    full_data_labels = []
    
    # Concatenate all training and test files into a single dataset
    for i in range(TRAIN_FILES):
        data_sets = input_data.read_data_sets(os.path.join(DATA_PATH, f'Train_{IMAGE_SIZE}_{i+1}.mat'), 'train')
        full_data_images.append(data_sets.images)
        full_data_labels.append(data_sets.labels)
    
    for i in range(TEST_FILES):
        data_sets = input_data.read_data_sets(os.path.join(DATA_PATH, f'Test_{IMAGE_SIZE}_{i+1}.mat'), 'test')
        full_data_images.append(data_sets.images)
        full_data_labels.append(data_sets.labels)
    
    # Combine all images and labels
    full_data_images = np.concatenate(full_data_images, axis=0)
    full_data_labels = np.concatenate(full_data_labels, axis=0)

    # full_data_labels = full_data_labels.flatten()
    
    # Reshape and preprocess the images
    full_data_images = full_data_images.reshape(full_data_images.shape[0], IMAGE_SIZE, IMAGE_SIZE, 220)
    full_data_images = np.transpose(full_data_images, (0, 3, 1, 2))  # Transpose to match input format
    
    # Initialize K-Fold Cross-Validation
    kf = KFold(n_splits=k_folds, shuffle=True, random_state=42)
    fold_accuracies = []
    
    for fold, (train_index, val_index) in enumerate(kf.split(full_data_images)):
        print(f"\nStarting Fold {fold + 1}/{k_folds}")
        
        # Split the data into training and validation sets
        train_images, val_images = full_data_images[train_index], full_data_images[val_index]
        train_labels, val_labels = full_data_labels[train_index], full_data_labels[val_index]
        
        # Create DataSet objects for training and validation
        Training_data = input_data.DataSet(train_images, train_labels)
        Validation_data = input_data.DataSet(val_images, val_labels)
        
        # Build the TensorFlow graph
        with tf.Graph().as_default():
            images_placeholder, labels_placeholder = placeholder_inputs(batch_size)
            logits = IndianPinesCNN.inference(images_placeholder, conv1, conv2, hidden1, hidden2)
            loss = IndianPinesCNN.loss(logits, labels_placeholder)
            train_op = IndianPinesCNN.training(loss, learning_rate)
            eval_correct = IndianPinesCNN.evaluation(logits, labels_placeholder)
            init = tf.global_variables_initializer()
            saver = tf.train.Saver()
            
            with tf.Session() as sess:
                sess.run(init)
                
                # Training loop
                for step in range(max_steps):
                    feed_dict = fill_feed_dict(Training_data, images_placeholder, labels_placeholder)
                    _, loss_value = sess.run([train_op, loss], feed_dict=feed_dict)
                    
                    if step % 50 == 0:
                        print(f"Step {step}: loss = {loss_value:.2f}")
                
                # Evaluate on the validation set
                print("Validation Data Eval:")
                true_count = 0
                steps_per_epoch = Validation_data.num_examples // batch_size
                num_examples = steps_per_epoch * batch_size
                
                for step in range(steps_per_epoch):
                    feed_dict = fill_feed_dict(Validation_data, images_placeholder, labels_placeholder)
                    true_count += sess.run(eval_correct, feed_dict=feed_dict)
                
                accuracy = true_count / num_examples
                print(f"Fold {fold + 1} Accuracy: {accuracy:.4f}")
                fold_accuracies.append(accuracy)
    
    # Print the average accuracy across all folds
    print(f"\nCross-Validation Accuracy: {np.mean(fold_accuracies):.4f} ± {np.std(fold_accuracies):.4f}")

In [11]:
run_training()

Tensor("conv_1/Conv2D:0", shape=(100, 19, 19, 500), dtype=float32)

Tensor("h_conv2/h_conv2:0", shape=(100, 8, 8, 100), dtype=float32)
Tensor("h_pool2:0", shape=(100, 4, 4, 100), dtype=float32)
Tensor("Reshape:0", shape=(100, 1600), dtype=float32)
Instructions for updating:
Use `tf.cast` instead.

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Use `tf.global_variables_initializer` instead.
Step 0: loss = 2.74 (0.736 sec)
Step 50: loss = 2.73 (0.546 sec)
Step 100: loss = 2.57 (0.551 sec)
Step 150: loss = 1.89 (0.612 sec)
Step 200: loss = 1.22 (0.720 sec)
Step 250: loss = 1.10 (0.717 sec)
Step 300: loss = 0.94 (0.748 sec)
Step 350: loss = 0.92 (0.725 sec)
Step 400: loss = 0.66 (0.706 sec)
Step 450: loss = 0.58 (0.723 sec)
Step 500: loss = 0.58 (0.799 sec)
Step 550: loss = 0.57 (0.919 sec)
Step 600: loss = 0.61 (0.703 sec)
Step 650: loss = 0.42 (0.704 sec)
Step 700: loss = 0.34 (0.700 sec)
St

In [16]:
# Run the training with cross-validation
# run_training_with_cross_validation(k_folds=5)